In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/Users/being/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='mps:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1958,  0.1092, -0.6082, -0.0336,  0.1120, -0.0724,  0.1716, -0.6608,
         -0.5123, -0.1960,  0.0058, -0.0167,  0.0302,  0.0512, -0.0909, -0.2167,
         -0.2022, -0.4176,  0.4306, -0.3794],
        [-0.2378,  0.2090, -0.9641, -0.2457, -0.0890,  0.3372,  0.1235, -0.2460,
         -0.5616, -0.1902, -0.1949, -0.0404, -0.2233, -0.1155,  0.1683, -0.7050,
         -0.3621, -0.0202,  0.4502, -0.0660],
        [-0.2998,  0.1450, -0.9555, -0.0820,  0.1186, -0.1440,  0.2609, -0.2468,
         -0.8329, -0.1625, -0.1876, -0.2235,  0.2383,  0.0638,  0.2221, -0.4494,
         -0.5170, -0.5671,  0.3247,  0.2116]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1092, 0.0000, 0.0000, 0.1120, 0.0000, 0.1716, 0.0000, 0.0000,
         0.0000, 0.0058, 0.0000, 0.0302, 0.0512, 0.0000, 0.0000, 0.0000, 0.0000,
         0.4306, 0.0000],
        [0.0000, 0.2090, 0.0000, 0.0000, 0.0000, 0.3372, 0.1235, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.16

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0185,  0.0213,  0.0216,  ...,  0.0125,  0.0057,  0.0013],
        [-0.0044,  0.0283,  0.0016,  ...,  0.0120,  0.0248,  0.0044]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0331, -0.0051], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0104,  0.0411, -0.0286,  ..., -0.0006, -0.0032,  0.0429],
        [-0.0089,  0.0410,  0.0005,  ..., -0.0056,  0.0383,  0.0090]],
       device='mps:0', grad_fn=<Slice